In [49]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

from tsfresh.examples import load_robot_execution_failures
import pandas as pd
import tsfresh
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier

from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from ML.feature_selection import FeatureSelection
from ML.model import Model, min_acc
from ML.process_data import Data
from utils.help_classes import ScalerEnum, ModelsEnum
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler

from utils.help_classes import ModelsEnum
from sklearn.model_selection import train_test_split
import os
import datetime
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

import pickle
import numpy as np
import category_encoders as ce
import plotly_express as px
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors, svm, metrics

from numpy.linalg import LinAlgError

In [2]:
X = pd.read_csv(os.path.join('data_csv', 'features_train.csv'))
X['time_stamp'] = pd.to_datetime(X['time_stamp'])

X_train = X.loc[X['time_stamp'] < X['time_stamp'].quantile(q=0.9)]
X_val = X.loc[X['time_stamp'] >= X['time_stamp'].quantile(q=0.9)]

In [3]:
y_train = X_train['label']
X_train = X_train.drop(columns=['label', 'time_stamp', 'traj'])

y_val = X_val['label']
X_val = X_val.drop(columns=['label', 'time_stamp', 'traj'])

In [4]:
print('before sampling', X_train.shape)
X_train = X_train.fillna(X_train.mean())

sampling_strategy = "not minority"
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
X_train, y_train = rus.fit_resample(X_train, y_train)
print('after sampling', X_train.shape)

before sampling (29466, 53)
after sampling (6381, 53)


In [5]:
X_train = X_train.fillna(X_train.median())
X_val = X_val.fillna(X_val.median())

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [44]:
class_weight = {0: 0.3, 1: 5, 2: 2}

In [58]:
level0 = list()
level0.append(('cart', DecisionTreeClassifier(class_weight=class_weight)))
level0.append(('svm', SVC(class_weight=class_weight)))
level0.append(('bayes', GaussianNB()))
level0.append(('rf', RandomForestClassifier(random_state=32, class_weight=class_weight)))
level0.append(('ab', AdaBoostClassifier(algorithm="SAMME", random_state=32)))
# level0.append(('xgb', xgb.XGBClassifier(use_label_encoder=False, random_state=32)))
# level0.append(('cb', CatBoostClassifier(iterations=1000)))
# level0.append(('gb', GradientBoostingClassifier()))

level1 = []
level1.append(('gb_1', StackingClassifier(estimators=level0, final_estimator=GradientBoostingClassifier(), cv=5)))
level1.append(('cb_1', StackingClassifier(estimators=level0, final_estimator=CatBoostClassifier(iterations=1000), cv=5)))
level1.append(('xgb_1', StackingClassifier(estimators=level0, 
                                 final_estimator=xgb.XGBClassifier(use_label_encoder=False, random_state=32), cv=5)))

model = StackingClassifier(estimators=level1, final_estimator=LogisticRegression(), cv=5)
model.fit(X_train, y_train)

Learning rate set to 0.086971
0:	learn: 0.9567317	total: 24ms	remaining: 24s
1:	learn: 0.8488302	total: 30.7ms	remaining: 15.3s
2:	learn: 0.7612618	total: 37.5ms	remaining: 12.5s
3:	learn: 0.6879226	total: 44.2ms	remaining: 11s
4:	learn: 0.6263488	total: 51.4ms	remaining: 10.2s
5:	learn: 0.5729577	total: 58.1ms	remaining: 9.63s
6:	learn: 0.5264985	total: 64.9ms	remaining: 9.21s
7:	learn: 0.4870873	total: 71.7ms	remaining: 8.89s
8:	learn: 0.4525446	total: 78.8ms	remaining: 8.68s
9:	learn: 0.4218312	total: 85.4ms	remaining: 8.46s
10:	learn: 0.3935410	total: 93.5ms	remaining: 8.4s
11:	learn: 0.3688529	total: 103ms	remaining: 8.48s
12:	learn: 0.3474343	total: 111ms	remaining: 8.39s
13:	learn: 0.3284318	total: 117ms	remaining: 8.26s
14:	learn: 0.3100657	total: 124ms	remaining: 8.16s
15:	learn: 0.2937754	total: 131ms	remaining: 8.04s
16:	learn: 0.2795249	total: 138ms	remaining: 7.98s
17:	learn: 0.2663663	total: 145ms	remaining: 7.89s
18:	learn: 0.2555541	total: 151ms	remaining: 7.8s
19:	lear

168:	learn: 0.0850593	total: 1.27s	remaining: 6.24s
169:	learn: 0.0847210	total: 1.28s	remaining: 6.24s
170:	learn: 0.0844329	total: 1.28s	remaining: 6.23s
171:	learn: 0.0841506	total: 1.29s	remaining: 6.21s
172:	learn: 0.0839953	total: 1.3s	remaining: 6.2s
173:	learn: 0.0837860	total: 1.3s	remaining: 6.19s
174:	learn: 0.0835971	total: 1.31s	remaining: 6.18s
175:	learn: 0.0833920	total: 1.32s	remaining: 6.17s
176:	learn: 0.0831986	total: 1.32s	remaining: 6.16s
177:	learn: 0.0830544	total: 1.33s	remaining: 6.15s
178:	learn: 0.0826576	total: 1.34s	remaining: 6.14s
179:	learn: 0.0824758	total: 1.35s	remaining: 6.13s
180:	learn: 0.0822002	total: 1.35s	remaining: 6.13s
181:	learn: 0.0820255	total: 1.36s	remaining: 6.13s
182:	learn: 0.0817752	total: 1.37s	remaining: 6.12s
183:	learn: 0.0816006	total: 1.38s	remaining: 6.11s
184:	learn: 0.0813427	total: 1.38s	remaining: 6.1s
185:	learn: 0.0812296	total: 1.39s	remaining: 6.08s
186:	learn: 0.0809228	total: 1.4s	remaining: 6.07s
187:	learn: 0.080

346:	learn: 0.0576996	total: 2.57s	remaining: 4.83s
347:	learn: 0.0576449	total: 2.58s	remaining: 4.82s
348:	learn: 0.0575773	total: 2.58s	remaining: 4.82s
349:	learn: 0.0574451	total: 2.59s	remaining: 4.81s
350:	learn: 0.0572503	total: 2.6s	remaining: 4.8s
351:	learn: 0.0571486	total: 2.6s	remaining: 4.8s
352:	learn: 0.0570843	total: 2.61s	remaining: 4.79s
353:	learn: 0.0569823	total: 2.62s	remaining: 4.78s
354:	learn: 0.0568677	total: 2.63s	remaining: 4.77s
355:	learn: 0.0567915	total: 2.63s	remaining: 4.76s
356:	learn: 0.0566576	total: 2.64s	remaining: 4.76s
357:	learn: 0.0565503	total: 2.65s	remaining: 4.75s
358:	learn: 0.0563983	total: 2.65s	remaining: 4.74s
359:	learn: 0.0562855	total: 2.66s	remaining: 4.73s
360:	learn: 0.0560787	total: 2.67s	remaining: 4.72s
361:	learn: 0.0559319	total: 2.67s	remaining: 4.71s
362:	learn: 0.0558083	total: 2.68s	remaining: 4.71s
363:	learn: 0.0557073	total: 2.69s	remaining: 4.7s
364:	learn: 0.0556366	total: 2.7s	remaining: 4.7s
365:	learn: 0.05555

518:	learn: 0.0431368	total: 3.83s	remaining: 3.55s
519:	learn: 0.0431053	total: 3.83s	remaining: 3.54s
520:	learn: 0.0430440	total: 3.84s	remaining: 3.53s
521:	learn: 0.0429814	total: 3.85s	remaining: 3.52s
522:	learn: 0.0428753	total: 3.86s	remaining: 3.52s
523:	learn: 0.0427982	total: 3.87s	remaining: 3.51s
524:	learn: 0.0427209	total: 3.88s	remaining: 3.51s
525:	learn: 0.0426597	total: 3.88s	remaining: 3.5s
526:	learn: 0.0425954	total: 3.89s	remaining: 3.49s
527:	learn: 0.0425062	total: 3.9s	remaining: 3.48s
528:	learn: 0.0424432	total: 3.9s	remaining: 3.48s
529:	learn: 0.0423475	total: 3.91s	remaining: 3.47s
530:	learn: 0.0422592	total: 3.92s	remaining: 3.46s
531:	learn: 0.0421763	total: 3.92s	remaining: 3.45s
532:	learn: 0.0421312	total: 3.93s	remaining: 3.44s
533:	learn: 0.0420769	total: 3.94s	remaining: 3.44s
534:	learn: 0.0420123	total: 3.95s	remaining: 3.43s
535:	learn: 0.0419646	total: 3.96s	remaining: 3.42s
536:	learn: 0.0418950	total: 3.96s	remaining: 3.42s
537:	learn: 0.0

692:	learn: 0.0337019	total: 5.11s	remaining: 2.26s
693:	learn: 0.0336314	total: 5.12s	remaining: 2.26s
694:	learn: 0.0335817	total: 5.13s	remaining: 2.25s
695:	learn: 0.0335510	total: 5.13s	remaining: 2.24s
696:	learn: 0.0335204	total: 5.14s	remaining: 2.23s
697:	learn: 0.0334841	total: 5.14s	remaining: 2.23s
698:	learn: 0.0334386	total: 5.15s	remaining: 2.22s
699:	learn: 0.0334125	total: 5.16s	remaining: 2.21s
700:	learn: 0.0333676	total: 5.17s	remaining: 2.2s
701:	learn: 0.0333149	total: 5.17s	remaining: 2.19s
702:	learn: 0.0332603	total: 5.18s	remaining: 2.19s
703:	learn: 0.0332236	total: 5.19s	remaining: 2.18s
704:	learn: 0.0331746	total: 5.2s	remaining: 2.17s
705:	learn: 0.0331403	total: 5.21s	remaining: 2.17s
706:	learn: 0.0330979	total: 5.21s	remaining: 2.16s
707:	learn: 0.0330596	total: 5.22s	remaining: 2.15s
708:	learn: 0.0330239	total: 5.23s	remaining: 2.15s
709:	learn: 0.0329730	total: 5.23s	remaining: 2.14s
710:	learn: 0.0329407	total: 5.24s	remaining: 2.13s
711:	learn: 0.

...

685:	learn: 0.0348071	total: 4.72s	remaining: 2.16s
686:	learn: 0.0347515	total: 4.73s	remaining: 2.15s
687:	learn: 0.0347192	total: 4.74s	remaining: 2.15s
688:	learn: 0.0346620	total: 4.74s	remaining: 2.14s
689:	learn: 0.0346367	total: 4.75s	remaining: 2.13s
690:	learn: 0.0345778	total: 4.76s	remaining: 2.13s
691:	learn: 0.0345449	total: 4.76s	remaining: 2.12s
692:	learn: 0.0345197	total: 4.77s	remaining: 2.11s
693:	learn: 0.0344603	total: 4.78s	remaining: 2.11s
694:	learn: 0.0344084	total: 4.78s	remaining: 2.1s
695:	learn: 0.0343473	total: 4.79s	remaining: 2.09s
696:	learn: 0.0343082	total: 4.8s	remaining: 2.08s
697:	learn: 0.0342644	total: 4.8s	remaining: 2.08s
698:	learn: 0.0342213	total: 4.81s	remaining: 2.07s
699:	learn: 0.0341696	total: 4.82s	remaining: 2.06s
700:	learn: 0.0341114	total: 4.82s	remaining: 2.06s
701:	learn: 0.0340731	total: 4.83s	remaining: 2.05s
702:	learn: 0.0340377	total: 4.84s	remaining: 2.04s
703:	learn: 0.0340136	total: 4.84s	remaining: 2.04s
704:	learn: 0.0

860:	learn: 0.0274517	total: 6.03s	remaining: 973ms
861:	learn: 0.0274230	total: 6.04s	remaining: 966ms
862:	learn: 0.0274015	total: 6.04s	remaining: 959ms
863:	learn: 0.0273544	total: 6.05s	remaining: 952ms
864:	learn: 0.0273217	total: 6.05s	remaining: 945ms
865:	learn: 0.0272775	total: 6.06s	remaining: 938ms
866:	learn: 0.0272372	total: 6.07s	remaining: 931ms
867:	learn: 0.0271901	total: 6.08s	remaining: 924ms
868:	learn: 0.0271292	total: 6.08s	remaining: 917ms
869:	learn: 0.0271054	total: 6.09s	remaining: 910ms
870:	learn: 0.0270895	total: 6.1s	remaining: 903ms
871:	learn: 0.0270566	total: 6.1s	remaining: 896ms
872:	learn: 0.0270301	total: 6.11s	remaining: 889ms
873:	learn: 0.0269935	total: 6.12s	remaining: 882ms
874:	learn: 0.0269438	total: 6.12s	remaining: 875ms
875:	learn: 0.0269075	total: 6.13s	remaining: 868ms
876:	learn: 0.0268833	total: 6.13s	remaining: 861ms
877:	learn: 0.0268617	total: 6.14s	remaining: 853ms
878:	learn: 0.0268377	total: 6.15s	remaining: 846ms
879:	learn: 0.

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[19:23:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:24:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:25:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:25:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

KeyboardInterrupt: 

In [51]:
y_pred = model.predict(X_val)

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


In [52]:
y_true_0 = y_val[y_val == 0]
y_pred_0 = y_pred[y_val == 0]

y_true_1 = y_val[y_val == 1]
y_pred_1 = y_pred[y_val == 1]

y_true_2 = y_val[y_val == 2]
y_pred_2 = y_pred[y_val == 2]

acc_0 = metrics.accuracy_score(y_true_0, y_pred_0)
acc_1 = metrics.accuracy_score(y_true_1, y_pred_1)
acc_2 = metrics.accuracy_score(y_true_2, y_pred_2)

results_dict = {
    "accuracy": [metrics.accuracy_score(y_val, y_pred)],
    'min_acc': [min_acc(y_val, y_pred)],
    'acc_0': [acc_0],
    'acc_1': [acc_1],
    'acc_2': [acc_2],
}

In [53]:
results_dict

{'accuracy': [0.9569465648854962],
 'min_acc': [0.9154589371980676],
 'acc_0': [0.9704697986577181],
 'acc_1': [0.9154589371980676],
 'acc_2': [0.9764150943396226]}

## Preparing submission

In [54]:
X_test = pd.read_csv(os.path.join('data_csv', 'features_test.csv'))

traj = X_test['traj'].values

X_test = X_test.drop(columns=['label', 'time_stamp', 'traj'])
X_test = X_test.fillna(X_test.median())
X_test = scaler.transform(X_test)

In [55]:
final_prediction = model.predict(X_test)

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


In [56]:
test_predictions = pd.DataFrame({
        'trajectory_ind': traj,
        'label': final_prediction
    })
test_predictions.to_csv('submission_3.csv', index=False)